***Extractive step: BioBERT, K means***

pipeline:
I use BioBERT for embdeddings

I use Spacy for sentence segmentation 



In [36]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.cluster import KMeans
from datasets import load_dataset
import pandas as pd
import time

import spacy
# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")


# Check if MPS is available and set the device accordingly
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) device.")
else:
    device = torch.device("cuda")
    print("MPS not available. Using GPU.")
    

# Load the dataset and cut down 
dataset = load_dataset("allenai/mslr2022", "ms2", split='train')
# Use select to create a subset
# dataset = dataset.select(range(20,30))  


# Initialize the BioBERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')
# model = BertModel.from_pretrained('dmis-lab/biobert-base-cased-v1.1').to(device)
tokenizer = BertTokenizer.from_pretrained('./biobert-v1.1')
model = BertModel.from_pretrained('./biobert-v1.1').to(device)


def bert_sentence_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        # Move inputs to the device
        inputs = tokenizer(sentence, return_tensors='pt', max_length=512, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        # Move outputs back to CPU
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy())
    return np.array(embeddings)

#this is fixed : each selected sentence is from a different cluster
def select_top_sentences(sentences, embeddings, n_sentences=5):
    if len(sentences) <= n_sentences:
        return ' '.join(sentences)

    kmeans = KMeans(n_clusters=n_sentences, n_init=10)
    kmeans.fit(embeddings)

    # Initialize a set to store indices of selected sentences
    selected_indices = set()
    top_sentences = []

    for i in range(n_sentences):
        # Calculate distances of all sentences from the i-th centroid
        distances = np.linalg.norm(embeddings - kmeans.cluster_centers_[i], axis=1)

        # Sort the sentences by their distance from the centroid
        sorted_indices = np.argsort(distances)

        # Find the closest sentence that hasn't been selected yet
        for index in sorted_indices:
            if index not in selected_indices:
                selected_indices.add(index)
                top_sentences.append(sentences[index])
                break

    return ' '.join(top_sentences)


def process_row(row):
    review_id = row['review_id']
    abstract_list = row['abstract']
    combined_summary = ''

    for abstract in abstract_list:
        # Check if the abstract is a string; if not, join it into a single string
        abstract_text = ' '.join(abstract) if isinstance(abstract, list) else abstract

        # Use SpaCy for sentence segmentation
        doc = nlp(abstract_text)
        sentences = [sent.text.strip() for sent in doc.sents]
        start = time.time()
        # Generate embeddings for each sentence
        embeddings = bert_sentence_embeddings(sentences)
        # Select the top sentences from these embeddings
        summary = select_top_sentences(sentences, embeddings)
        print(time.time() - start)
        # Combine the summaries from each abstract
        combined_summary += summary + ' '

    return {"review_id": review_id, "summary": combined_summary.strip()}

# Apply the function to each element of the dataset
summaries_dataset = dataset.map(process_row)

# Saving the dataset
# import pickle
# with open('summaries_dataset.pkl', 'wb') as file:
#     pickle.dump(summaries_dataset, file)

# Convert to pandas DataFrame
df = pd.DataFrame(summaries_dataset)
df = df[['review_id', 'summary']]
# Save to CSV
csv_file_path = 'embeding_1.csv'  # Update with your desired file path
df.to_csv(csv_file_path, index=True)

print(f"Saved summaries to {csv_file_path}")

MPS not available. Using GPU.


Map:   0%|          | 0/14188 [00:00<?, ? examples/s]

0.08138918876647949
0.09297895431518555
0.04000544548034668
0.28267478942871094
0.0876317024230957
0.29001903533935547
0.08760666847229004
0.4520113468170166
0.09695553779602051
0.3567330837249756
0.39965367317199707
0.08008861541748047
0.08632183074951172
0.27204465866088867


Map:   0%|          | 1/14188 [00:03<14:13:50,  3.61s/ examples]

0.22082042694091797
0.17784595489501953
0.2112441062927246
0.1045384407043457
0.21747064590454102


...

0.41775083541870117
0.4109306335449219
0.24704480171203613
0.2847890853881836
0.3306314945220947
0.49283504486083984
0.4492044448852539
0.018002748489379883
0.08524751663208008
0.3352806568145752
0.37081432342529297
0.34940361976623535
0.4293384552001953
0.3294670581817627
0.3865063190460205
0.4703538417816162


Map:   7%|▋         | 1019/14188 [1:57:09<20:02:54,  5.48s/ examples]

0.4828355312347412
0.505284309387207
0.2251131534576416
0.3217196464538574
0.44344234466552734
0.4238860607147217
0.38294076919555664
0.4161694049835205
0.07938146591186523
0.4429762363433838
0.4304370880126953
0.2391071319580078
0.22822308540344238
0.23978710174560547
0.37876105308532715
0.15666985511779785
0.28704071044921875
0.10816836357116699
0.1963651180267334
0.09906125068664551
0.09338235855102539
0.07863330841064453
0.06118130683898926


Map:   7%|▋         | 1019/14188 [1:57:16<25:15:36,  6.91s/ examples]


KeyboardInterrupt: 

***abstractive step: Long T5***

this is without batching

In [ ]:
# import gc
# from transformers import AutoTokenizer, LongT5ForConditionalGeneration
# import pandas as pd
# import torch
# from torch.cuda.amp import autocast
# from datasets import load_dataset


# # Set the device to MPS if available, else CPU
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# # Load LongT5 Model and Tokenizer
# model_to_use = "pszemraj/long-t5-tglobal-base-16384-book-summary"  # fine-tuned for summarization
# longt5_model = LongT5ForConditionalGeneration.from_pretrained(model_to_use).to(device)
# longt5_tokenizer = AutoTokenizer.from_pretrained(model_to_use)


# # Dictionary to store the final summaries
# final_summaries = {}

# # Iterate over each summary in the summaries dataset
# for row in summaries_dataset:
#     review_id = row['review_id']
#     extractive_summary = row['summary']

#     # Prepare the input for the model
#     inputs = longt5_tokenizer(
#         extractive_summary, 
#         truncation=True, 
#         padding="longest", 
#         return_tensors="pt", 
#         max_length=16384
#     ).to(device)

#     # Generate the summary with LongT5
#     try:
#         summary_ids = longt5_model.generate(
#             inputs['input_ids'], 
#             num_beams=4,
#             min_length=50,
#             max_length=400,             
#             length_penalty=2.0, 
#             early_stopping=True
#         )
        
#         # Decode the generated IDs to text
#         longt5_summary = longt5_tokenizer.decode(
#             summary_ids[0], 
#             skip_special_tokens=True
#         )
        
#         # Store the summary in the final summaries dictionary
#         final_summaries[review_id] = longt5_summary

#     except IndexError as e:
#         print(f"Error processing review_id {review_id}: {e}")
#         final_summaries[review_id] = ""

# # Display the final summaries
# for review_id, summary in final_summaries.items():
#     print(f"Review ID: {review_id}\nAbstractive Summary: {summary}\n")


In [ ]:
# import pickle

# # Saving the dataset
# # with open('summaries_dataset.pkl', 'wb') as file:
# #     pickle.dump(summaries_dataset, file)

# # Later, you can load the dataset
# with open('summaries_dataset.pkl', 'rb') as file:
#     summaries_dataset = pickle.load(file)


batching version 1

In [ ]:
# from transformers import AutoTokenizer, LongT5ForConditionalGeneration
# import pandas as pd
# import torch
# from torch.cuda.amp import autocast
# from datasets import load_dataset


# # Check if MPS is available and set the device accordingly
# if torch.backends.mps.is_available():
#     device = torch.device("mps")
#     print("Using MPS (Metal Performance Shaders) device.")
# else:
#     device = torch.device("cpu")
#     print("MPS not available. Using CPU.")

# # Load LongT5 Model and Tokenizer
# model_to_use = "pszemraj/long-t5-tglobal-base-16384-book-summary"  # fine-tuned for summarization
# longt5_model = LongT5ForConditionalGeneration.from_pretrained(model_to_use).to(device)
# longt5_tokenizer = AutoTokenizer.from_pretrained(model_to_use)

# # Define the batch size
# batch_size = 1

# # Initialize the results list
# results = []

# # Convert the dataset to a list of dictionaries if not already
# data_list = list(summaries_dataset)

# # Generate summaries in batches
# for i in range(0, len(data_list), batch_size):
#     batch = data_list[i:i + batch_size]
#     input_texts = [row['summary'] for row in batch]
#     review_ids_batch = [row['review_id'] for row in batch]

#     inputs = longt5_tokenizer(
#         input_texts,
#         truncation=True,
#         padding="longest",
#         return_tensors="pt",
#         max_length=16384
#     ).to(device)

#     try:
#         summary_ids = longt5_model.generate(
#             inputs['input_ids'],
#             num_beams=2,
#             no_repeat_ngram_size=2,
#             min_length=10,
#             max_length=512,
#             early_stopping=True
#         ).to('cpu')
        
#         batch_summaries = longt5_tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

#         # Append each summary with its ReviewID to the results list
#         for review_id, summary in zip(review_ids_batch, batch_summaries):
#             results.append({'review_id': review_id, 'Summary': summary})

#     except Exception as e:
#         print(f"Error in batch starting at index {i}: {e}")
#         for review_id in review_ids_batch:
#             results.append({'review_id': review_id, 'Summary': ""})

# # Convert the results to a DataFrame
# summaries_df_val = pd.DataFrame(results)

# # Save the DataFrame to a CSV file
# output_file_val = 'summary_BioBERT_K_Means_Long_T5_prediction.csv'
# summaries_df_val.to_csv(output_file_val, index=True)
# print(f"Saved summaries to {output_file_val}")

#batching version 2: using a method that saves summaries and clears memory

In [ ]:
# from transformers import AutoTokenizer, LongT5ForConditionalGeneration
# import pandas as pd
# import torch
# import os
# import gc  # For garbage collection
# import pickle


# # Check for device availability and set accordingly
# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     print("Using CUDA device.")
# elif torch.backends.mps.is_available():
#     device = torch.device("mps")
#     print("Using MPS (Metal Performance Shaders) device.")
# else:
#     device = torch.device("cpu")
#     print("CUDA and MPS not available. Using CPU.")


# # Load LongT5 Model and Tokenizer
# model_to_use = "pszemraj/long-t5-tglobal-base-16384-book-summary"  # fine-tuned for summarization
# longt5_model = LongT5ForConditionalGeneration.from_pretrained(model_to_use).to(device)
# longt5_tokenizer = AutoTokenizer.from_pretrained(model_to_use)

# # Define the batch size and save directory
# batch_size = 1
# save_dir = 'summaries'  # Directory to save individual summary files
# os.makedirs(save_dir, exist_ok=True)

# # Initialize the results list
# results = []



# #loading the .pkl file if needed 
# # you can load the dataset
# with open('summaries_dataset.pkl', 'rb') as file:
#     summaries_dataset = pickle.load(file)



# # Convert the dataset to a list of dictionaries if not already
# data_list = list(summaries_dataset)

# #Select a subset of the dataset if needed
# data_list = data_list[1000:1020]


# # Generate summaries in batches
# for i in range(0, len(data_list), batch_size):
#     batch = data_list[i:i + batch_size]
#     input_texts = [row['summary'] for row in batch]
#     review_ids_batch = [row['review_id'] for row in batch]

#     inputs = longt5_tokenizer(
#         input_texts,
#         truncation=True,
#         padding="longest",
#         return_tensors="pt",
#         max_length=16384
#     ).to(device)

#     try:
#         summary_ids = longt5_model.generate(
#             inputs['input_ids'],
#             num_beams=2,
#             no_repeat_ngram_size=2,
#             min_length=10,
#             max_length=512,
#             early_stopping=True
#         ).to('cpu')
        
#         batch_summaries = longt5_tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

#         # Save summaries and append to results
#         for review_id, summary in zip(review_ids_batch, batch_summaries):
#             with open(os.path.join(save_dir, f"{review_id}.txt"), "w") as f:
#                 f.write(summary)
#             results.append({'review_id': review_id, 'Summary': summary})

#     except Exception as e:
#         print(f"Error in batch starting at index {i}: {e}")
#         for review_id in review_ids_batch:
#             results.append({'review_id': review_id, 'Summary': ""})

#     # Clear memory
#     del inputs, summary_ids
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#     elif torch.backends.mps.is_available():
#         torch.mps.empty_cache()
#     gc.collect()

# # Convert the results to a DataFrame
# summaries_df_val = pd.DataFrame(results)

# # Define the destination folder
# output_folder = 'summaries'

# # Define the CSV file name within the folder
# output_file_val = os.path.join(output_folder, 'summary_BioBERT_K_Means_Long_T5_prediction.csv')
# # Save the DataFrame to the CSV file
# summaries_df_val.to_csv(output_file_val, index=True)
# print(f"Saved summaries to {output_file_val}")


In [1]:
!pip -q install torch
!pip install -q --upgrade transformers datasets sentencepiece
!pip install -q transformers datasets sentencepiece rouge_score
import torch
print(torch.__version__)

DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the aut

In [5]:
!ls
!pwd


 266_Final_project-T5_Amir.ipynb	   distilbert-base-uncased
 266_Final_project-farid.ipynb		   distilbert-base-uncased.zip
 266_Final_project.ipynb		   download_offline_hugging.ipynb
 266_Final_project_T5_Amir_BookSum.ipynb   extractive_models.ipynb
 BioBERT_HDBSCAN_UMAP_Long_T5.ipynb	   lightning_logs
 BioBERT_K_Means_Long_T5.ipynb		   logs
 Fine_Tuning_T5_example.ipynb		   longt5_finetuned
 README.md				   longt5_ms2_finetuned
'T5Long_Fine_tune _1.ipynb'		   longt5_ms2_results
 T5Long_Fine_tune.ipynb			   model_evaluation_output.csv
 bert					   ms2
 biobert				   results
 biobert-v1				   summaries_dataset.pkl
 biobert-v1.1
/mnt/wekamount/RI-Users/amir.moazami/Projects/266/266_final_proj


#version 3: instead of saving to a txt file, append to a csv file 

In [ ]:
from transformers import AutoTokenizer, LongT5ForConditionalGeneration
import pandas as pd
import torch
import os
import gc  # For garbage collection
import pickle
import csv


max_split_size_mb = 256  # Set the max_split_size_mb value (e.g., 512 MB)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = f"max_split_size_mb:{max_split_size_mb}"

# Check for device availability and set accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA device.")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) device.")
else:
    device = torch.device("cpu")
    print("CUDA and MPS not available. Using CPU.")

# Load LongT5 Model and Tokenizer
model_to_use = "pszemraj/long-t5-tglobal-base-16384-book-summary"  # Fine-tuned for summarization
longt5_model = LongT5ForConditionalGeneration.from_pretrained(model_to_use).to(device)
longt5_tokenizer = AutoTokenizer.from_pretrained(model_to_use)


# data_list = []

# Load the dataset from a pickle file
# with open('./summaries_dataset.pkl', 'rb') as file:
# with open('/mnt/wekamount/RI-Users/amir.moazami/Projects/266/266_final_proj/BioBERT_K_Means_extractive.csv', 'r') as file:
#     summaries_dataset = csv.DictReader(file)
#     data_list = list(summaries_dataset)

# Load the dataset from a CSV file
dataset_path = '/mnt/wekamount/RI-Users/amir.moazami/Projects/266/266_final_proj/BioBERT_K_Means_extractive.csv'
summaries_dataset = pd.read_csv(dataset_path , index_col=False)
# Convert the dataset to a list of dictionaries
data_list = summaries_dataset.to_dict(orient='records')
#############


# # Convert the dataset to a list
# data_list = list(summaries_dataset)

# # Select a subset of the dataset for processing
# data_list = data_list[1240:1255]

# Define the destination folder and CSV file name
output_folder = 'summaries'
os.makedirs(output_folder, exist_ok=True)
output_file_val = os.path.join(output_folder, 'summary_BioBERT_K_Means_Long_T5_prediction.csv')

# Check if the output file already exists, create it with headers if not
if not os.path.exists(output_file_val):
    pd.DataFrame(columns=['review_id', 'Summary']).to_csv(output_file_val, index=False)

# Define the batch size
batch_size = 1

# Generate summaries in batches
for i in range(0, len(data_list), batch_size):
    batch = data_list[i:i + batch_size]
    input_texts = [row['summary'] for row in batch]
    review_ids_batch = [row['review_id'] for row in batch]

    # Tokenize the input texts
    inputs = longt5_tokenizer(
        input_texts,
        truncation=True,
        padding="longest",
        return_tensors="pt",
        max_length=16384
    ).to(device)

    try:
        # Generate summaries
        summary_ids = longt5_model.generate(
            inputs['input_ids'],
            num_beams=2,
            no_repeat_ngram_size=2,
            min_length=10,
            max_length=512,
            early_stopping=True
        ).to('cpu')

        # Decode the summaries
        batch_summaries = longt5_tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

        # Append summaries directly to the CSV file
        with open(output_file_val, 'a') as f:
            pd.DataFrame({'review_id': review_ids_batch, 'Summary': batch_summaries}).to_csv(f, header=False, index=False)


    except Exception as e:
        print(f"Error in batch starting at index {i}: {e}")

    print(f"Index {i} done!")

    # Clear memory
    del inputs, summary_ids
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif torch.backends.mps.is_available():
        torch.mps.empty_cache()
    gc.collect()

print(f"Saved summaries to {output_file_val}. Abstractive summarization portion done!")

Using CUDA device.


/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/transformers/models/longt5/modeling_longt5.py:170: UserWarning: An output with one or more elements was resized since it had shape [4630], which does not match the required output shape [1, 4630]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  true_block_ends = torch.logical_and(block_ends, block_ids >= 0)
/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/transformers/models/longt5/modeling_longt5.py:146: UserWarning: An output with one or more elements was resized since it had shape [1, 37, 128, 1], which does not match the required output shape [1, 37, 128, 384]. This behavior is deprecated, and in a future PyTorch release output

Index 0 done!


/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/transformers/models/longt5/modeling_longt5.py:170: UserWarning: An output with one or more elements was resized since it had shape [2290], which does not match the required output shape [1, 2290]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  true_block_ends = torch.logical_and(block_ends, block_ids >= 0)
/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/transformers/models/longt5/modeling_longt5.py:146: UserWarning: An output with one or more elements was resized since it had shape [1, 18, 128, 1], which does not match the required output shape [1, 18, 128, 384]. This behavior is deprecated, and in a future PyTorch release output

Index 1 done!


...

/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/transformers/models/longt5/modeling_longt5.py:170: UserWarning: An output with one or more elements was resized since it had shape [2212], which does not match the required output shape [1, 2212]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  true_block_ends = torch.logical_and(block_ends, block_ids >= 0)


Index 17 done!


/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/transformers/models/longt5/modeling_longt5.py:170: UserWarning: An output with one or more elements was resized since it had shape [4962], which does not match the required output shape [1, 4962]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  true_block_ends = torch.logical_and(block_ends, block_ids >= 0)
/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/transformers/models/longt5/modeling_longt5.py:146: UserWarning: An output with one or more elements was resized since it had shape [1, 39, 128, 1], which does not match the required output shape [1, 39, 128, 384]. This behavior is deprecated, and in a future PyTorch release output

Index 18 done!


/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/transformers/models/longt5/modeling_longt5.py:170: UserWarning: An output with one or more elements was resized since it had shape [1621], which does not match the required output shape [1, 1621]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  true_block_ends = torch.logical_and(block_ends, block_ids >= 0)


#batching using huggingface module, map

In [ ]:
# import pandas as pd
# import torch
# from transformers import AutoTokenizer, LongT5ForConditionalGeneration
# from datasets import load_dataset

# # Check if MPS is available and set the device accordingly
# if torch.backends.mps.is_available():
#     device = torch.device("mps")
#     print("Using MPS (Metal Performance Shaders) device.")
# else:
#     device = torch.device("cpu")
#     print("MPS not available. Using CPU.")

# # Load LongT5 Model and Tokenizer
# model_to_use = "pszemraj/long-t5-tglobal-base-16384-book-summary"  # fine-tuned for summarization
# longt5_model = LongT5ForConditionalGeneration.from_pretrained(model_to_use).to(device)
# longt5_tokenizer = AutoTokenizer.from_pretrained(model_to_use)

# # Load your dataset here (assuming 'summaries_dataset' is your dataset name)
# # summaries_dataset = load_dataset('your_dataset_name')

# # Define the batch processing function
# def process_batch(batch):
#     input_texts = batch['summary']
#     review_ids_batch = batch['review_id']

#     inputs = longt5_tokenizer(
#         input_texts,
#         truncation=True,
#         padding="longest",
#         return_tensors="pt",
#         max_length=16384
#     ).to(device)

#     try:
#         summary_ids = longt5_model.generate(
#             inputs['input_ids'],
#             num_beams=4,
#             min_length=50,
#             max_length=512,
#             early_stopping=True
#         ).to('cpu')
        
#         batch_summaries = longt5_tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

#         # Return processed results
#         return {'review_id': review_ids_batch, 'Abstractive Summary': batch_summaries}

#     except Exception as e:
#         print(f"Error in processing batch: {e}")
#         # Returning empty summaries in case of error
#         return {'review_id': review_ids_batch, 'Abstractive Summary': ['' for _ in review_ids_batch]}

# # Apply the function to the entire dataset using map
# batch_size = 1  # Set your batch size
# processed_dataset = summaries_dataset.map(process_batch, batched=True, batch_size=batch_size)

# # Convert the processed results to a pandas DataFrame
# summaries_df = pd.DataFrame.from_dict({
#     'review_id': processed_dataset['review_id'],
#     'Abstractive Summary': processed_dataset['Abstractive Summary']
# })

# # Save the DataFrame to a CSV file
# output_file = 'summary_predictions.csv'
# summaries_df.to_csv(output_file, index=False)
# print(f"Saved summaries to {output_file}")
